### Download games for particular season

In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints.boxscoresummaryv2 import BoxScoreSummaryV2
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

SEASON = '2020-2021'

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
league_dict = [team for team in nba_teams]
nba_ids = [team['id'] for team in league_dict]


In [11]:
from nba_api.stats.endpoints import teamgamelog

season_games = pd.DataFrame()
for id in nba_ids:
    # Query for all games for all teams
    gamefinder = teamgamelog.TeamGameLog(team_id=id, season=SEASON, season_type_all_star='Regular Season')
    # The first DataFrame of those returned is what we want.
    games = gamefinder.get_data_frames()[0]
    print(len(games))
    # Filter for 2018 season and concat all games into one DataFrame
    season_games = season_games.append(games)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# All games from the 2018/19 season
season_games.head()

In [ ]:
# Store season games in a csv file
season_games.to_csv('season_games_20182019.csv', index=False)

### Download box scores for games

In [2]:
# Read in the csv file
season_games = pd.read_csv('season_games_20182019.csv',
                           converters={'Game_ID': str})
# Get game id column from the season games keeping leading zeros
game_ids = season_games['Game_ID'].values
print(len(game_ids))


2460


In [ ]:
season_df = pd.DataFrame()

for idx, id in enumerate(game_ids):
    box = box = BoxScoreSummaryV2(id)
    box_dict = box.get_dict()

    # Get headers and rows for each DataFrame
    other_stats_headers = box_dict['resultSets'][1]['headers']
    other_stats_rows = box_dict['resultSets'][1]['rowSet']
    line_score_headers = box_dict['resultSets'][5]['headers']
    line_score_rows = box_dict['resultSets'][5]['rowSet']

    # Combine the headers and rows into a DataFrame
    other_df = pd.DataFrame(other_stats_rows, columns=other_stats_headers)
    line_df = pd.DataFrame(line_score_rows, columns=line_score_headers)

    # Combine other_df and line_df into one DataFrame
    combined_df = pd.concat([other_df, line_df], axis=1)
    
    # Add to season_df
    season_df = season_df.append(combined_df)
    season_df.to_csv('season_box_scores_20182019_tmp.csv', index=False)

    print(idx)
